In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# path where the data is contained
path = '/content/drive/MyDrive/IHLT/lab2/'

import pandas as pd
import re
import nltk
from nltk import CFG, BottomUpChartParser, BottomUpLeftCornerChartParser, LeftCornerChartParser
from nltk.metrics import jaccard_distance
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from scipy.stats import pearsonr
from IPython.display import display_html

from nltk.parse.corenlp import CoreNLPDependencyParser

Mounted at /content/drive


# Lab 8: Parsing

For the eighth practical of the subject, the goal is to try some non-probabilistic parsers, and optionally probabilistic parsers as well. The **mandatory** statement is:

1. Consider the following sentence:
`Lazy cats play with mice.`
2. Expand the grammar of the example related to non-probabilistic chart parsers in order to subsume this new sentence.
3. Perform the constituency parsing using a BottomUpChartParser, a BottomUpLeftCornerChartParser and a LeftCornerChartParser.
4. For each one of them, provide the resulting tree, the number of edges and the list of explored edges.
5. Which parser is the most efficient for parsing the sentence?
6. Which edges are filtered out by each parser and why?

The **optional** statement, which we've also accomplished, is:

1. Read all pairs of sentences of the SMTeuroparl files of test set within the evaluation framework of the project.
2. Compute the Jaccard similarity of each pair using the dependency triples from CoreNLPDependencyParser.
3. Show the results. Do you think it could be relevant to use NEs to compute the similarity between two sentences? Justify the answer.

## Mandatory exercise: Non-probabilistic parsers

We add the words `"lazy"` (adjective, `Adj`), `"play"` (verb, `V`) and `"with"` (preposition, `PP`) in order to expand the grammar given so it satisfies the sentence `Lazy cats play with mice`. 

Reference: https://www.nltk.org/book/ch08.html

In [ ]:
grammar = CFG.fromstring('''
  NP  -> NNS | JJ NNS | NP CC NP
  NNS -> "cats" | "dogs" | "mice" | NNS CC NNS 
  JJ  -> "big" | "small"
  CC  -> "and" | "or"
  PP -> "with"
  V -> "play"
  Adj -> "lazy"
  ''')
sent = ['lazy', 'cats', 'play', 'with', 'mice']

### BottomUpChartParser

In [ ]:
parser = nltk.BottomUpChartParser(grammar,trace=1)

In [ ]:
parse = parser.chart_parse(sent)

|.  lazy .  cats .  play .  with .  mice .|
|[-------]       .       .       .       .| [0:1] 'lazy'
|.       [-------]       .       .       .| [1:2] 'cats'
|.       .       [-------]       .       .| [2:3] 'play'
|.       .       .       [-------]       .| [3:4] 'with'
|.       .       .       .       [-------]| [4:5] 'mice'
|>       .       .       .       .       .| [0:0] Adj -> * 'lazy'
|[-------]       .       .       .       .| [0:1] Adj -> 'lazy' *
|.       >       .       .       .       .| [1:1] NNS -> * 'cats'
|.       [-------]       .       .       .| [1:2] NNS -> 'cats' *
|.       >       .       .       .       .| [1:1] NP -> * NNS
|.       >       .       .       .       .| [1:1] NNS -> * NNS CC NNS
|.       [-------]       .       .       .| [1:2] NP -> NNS *
|.       [------->       .       .       .| [1:2] NNS -> NNS * CC NNS
|.       >       .       .       .       .| [1:1] NP -> * NP CC NP
|.       [------->       .       .       .| [1:2] NP -> NP * CC NP
|.       

In [ ]:
print("Number of edges:  " + str(parse.num_edges()))

Number of edges:  27


In [ ]:
parse.edges()

[[Edge: [0:1] 'lazy'],
 [Edge: [1:2] 'cats'],
 [Edge: [2:3] 'play'],
 [Edge: [3:4] 'with'],
 [Edge: [4:5] 'mice'],
 [Edge: [0:0] Adj -> * 'lazy'],
 [Edge: [0:1] Adj -> 'lazy' *],
 [Edge: [1:1] NNS -> * 'cats'],
 [Edge: [1:2] NNS -> 'cats' *],
 [Edge: [1:1] NP -> * NNS],
 [Edge: [1:1] NNS -> * NNS CC NNS],
 [Edge: [1:2] NP -> NNS *],
 [Edge: [1:2] NNS -> NNS * CC NNS],
 [Edge: [1:1] NP -> * NP CC NP],
 [Edge: [1:2] NP -> NP * CC NP],
 [Edge: [2:2] V  -> * 'play'],
 [Edge: [2:3] V  -> 'play' *],
 [Edge: [3:3] PP -> * 'with'],
 [Edge: [3:4] PP -> 'with' *],
 [Edge: [4:4] NNS -> * 'mice'],
 [Edge: [4:5] NNS -> 'mice' *],
 [Edge: [4:4] NP -> * NNS],
 [Edge: [4:4] NNS -> * NNS CC NNS],
 [Edge: [4:5] NP -> NNS *],
 [Edge: [4:5] NNS -> NNS * CC NNS],
 [Edge: [4:4] NP -> * NP CC NP],
 [Edge: [4:5] NP -> NP * CC NP]]

### BottomUpLeftCornerChartParser

In [ ]:
parser = nltk.BottomUpLeftCornerChartParser(grammar,trace=1)

In [ ]:
parse = parser.chart_parse(sent)

|.  lazy .  cats .  play .  with .  mice .|
|[-------]       .       .       .       .| [0:1] 'lazy'
|.       [-------]       .       .       .| [1:2] 'cats'
|.       .       [-------]       .       .| [2:3] 'play'
|.       .       .       [-------]       .| [3:4] 'with'
|.       .       .       .       [-------]| [4:5] 'mice'
|[-------]       .       .       .       .| [0:1] Adj -> 'lazy' *
|.       [-------]       .       .       .| [1:2] NNS -> 'cats' *
|.       [-------]       .       .       .| [1:2] NP -> NNS *
|.       [------->       .       .       .| [1:2] NNS -> NNS * CC NNS
|.       [------->       .       .       .| [1:2] NP -> NP * CC NP
|.       .       [-------]       .       .| [2:3] V  -> 'play' *
|.       .       .       [-------]       .| [3:4] PP -> 'with' *
|.       .       .       .       [-------]| [4:5] NNS -> 'mice' *
|.       .       .       .       [-------]| [4:5] NP -> NNS *
|.       .       .       .       [------->| [4:5] NNS -> NNS * CC NNS
|.       .  

In [ ]:
print("Number of edges:  " + str(parse.num_edges()))

Number of edges:  16


In [ ]:
parse.edges()

[[Edge: [0:1] 'lazy'],
 [Edge: [1:2] 'cats'],
 [Edge: [2:3] 'play'],
 [Edge: [3:4] 'with'],
 [Edge: [4:5] 'mice'],
 [Edge: [0:1] Adj -> 'lazy' *],
 [Edge: [1:2] NNS -> 'cats' *],
 [Edge: [1:2] NP -> NNS *],
 [Edge: [1:2] NNS -> NNS * CC NNS],
 [Edge: [1:2] NP -> NP * CC NP],
 [Edge: [2:3] V  -> 'play' *],
 [Edge: [3:4] PP -> 'with' *],
 [Edge: [4:5] NNS -> 'mice' *],
 [Edge: [4:5] NP -> NNS *],
 [Edge: [4:5] NNS -> NNS * CC NNS],
 [Edge: [4:5] NP -> NP * CC NP]]

### LeftCornerChartParser

In [ ]:
parser = nltk.LeftCornerChartParser(grammar,trace=1)

In [ ]:
parse = parser.chart_parse(sent)

|.  lazy .  cats .  play .  with .  mice .|
|[-------]       .       .       .       .| [0:1] 'lazy'
|.       [-------]       .       .       .| [1:2] 'cats'
|.       .       [-------]       .       .| [2:3] 'play'
|.       .       .       [-------]       .| [3:4] 'with'
|.       .       .       .       [-------]| [4:5] 'mice'
|[-------]       .       .       .       .| [0:1] Adj -> 'lazy' *
|.       [-------]       .       .       .| [1:2] NNS -> 'cats' *
|.       [-------]       .       .       .| [1:2] NP -> NNS *
|.       .       [-------]       .       .| [2:3] V  -> 'play' *
|.       .       .       [-------]       .| [3:4] PP -> 'with' *
|.       .       .       .       [-------]| [4:5] NNS -> 'mice' *
|.       .       .       .       [-------]| [4:5] NP -> NNS *


In [ ]:
print("Number of edges:  " + str(parse.num_edges()))

Number of edges:  12


In [ ]:
parse.edges()

[[Edge: [0:1] 'lazy'],
 [Edge: [1:2] 'cats'],
 [Edge: [2:3] 'play'],
 [Edge: [3:4] 'with'],
 [Edge: [4:5] 'mice'],
 [Edge: [0:1] Adj -> 'lazy' *],
 [Edge: [1:2] NNS -> 'cats' *],
 [Edge: [1:2] NP -> NNS *],
 [Edge: [2:3] V  -> 'play' *],
 [Edge: [3:4] PP -> 'with' *],
 [Edge: [4:5] NNS -> 'mice' *],
 [Edge: [4:5] NP -> NNS *]]

### **Conclusions**

**Which parser is the most efficient for parsing the sentence?**

**Which edges are filtered out by each parser and why?**

## Optional exercise: Dependency parser

To use, we first need to download and run the CoreNLP server on `localhost:9000` by following the next few steps:

1. Download CoreNLP at https://stanfordnlp.github.io/CoreNLP/download.html
2. Unzip the files and run the following command in the that directory to start the server: `java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000`


In [ ]:
parser = CoreNLPDependencyParser(url='http://localhost:9000/')
parse, = parser.raw_parse('Smith jumps over the lazy dog')
for governor, dep, dependent in parse.triples():
  print(governor, dep, dependent)

ConnectionError: ignored